In [10]:
# Idalia_radial_model_data_comp
import os
import pickle
from datetime import datetime, timezone
from scipy import stats
import matplotlib.pyplot as plt
import pandas as pd

from pathlib import Path

import matplotlib.colors
import matplotlib.cm as cm

import numpy as np

import xarray as xr
import xroms
import matplotlib.pyplot as plt
from wavespectra import read_ww3, read_swan, read_ndbc, read_netcdf
from wavespectra.input.swan import read_swans

%run -i wave_stats.py
%run -i spec_plot_funcs.py

#### Import Hurricane Track

In [2]:
import geopandas as gpd
import cartopy.crs as crs
import cartopy.feature as cf
gd1 = gpd.read_file(DATA_DIRECTORY+'track/AL102023_pts.shp')
lt1 = gd1.LAT
ln1 = gd1.LON
hurpth = np.array(list(zip(ln1,lt1)))

#### Load drifter data and assign nicknames

In [3]:
with open(os.path.join(DATA_DIRECTORY, DATA_FILENAME), 'rb') as handle:
    drifters = pickle.load(handle)

drifter_types = ['spotter', 'dwsd', 'microswift']
pfx = ['S', 'D', 'M']
# `drifters` is a python dictionary keyed by drifter type (spotter,
# dwsd, or microswift)
# `spotter` is a python dictionary of Pandas DataFrames, keyed by
# each drifter ID. The drifter ids can then be accessed as follows:
spotter = drifters['spotter']
spotter_ids = list(spotter.keys())

dwsd = drifters['dwsd']
dwsd_ids = list(dwsd.keys())
    
mswift = drifters['microswift']
mswift_ids = list(mswift.keys())

# Make nicknames for these drifters to conform with the max. 8 char limit in SWAN filesname
# after adding three characters for a counter (to differentiate the POINTS).
snames=['S025D','S052D','S055D','S061D','S066D','S095D','S101D','S102D','S103D','S164D',
    'D9690','D1280','D3160', 'D3730','D0060','D0070','D0090','D0250','D8160','D9490','D8010',
    'M0029','M0037','M0046','M0048']

icount=0
for dtype in drifter_types:
    drifter_data = drifters[dtype]
    for id in list(drifter_data.keys()):
        print(id, snames[icount])
        icount += 1

SPOT-30025D S025D
SPOT-30052D S052D
SPOT-30055D S055D
SPOT-30061D S061D
SPOT-30066D S066D
SPOT-30095D S095D
SPOT-30101D S101D
SPOT-30102D S102D
SPOT-30103D S103D
SPOT-30164D S164D
300534060649690 D9690
300534060941280 D1280
300534064703160 D3160
300534064703730 D3730
300534064800060 D0060
300534064800070 D0070
300534064800090 D0090
300534064800250 D0250
300534061518160 D8160
300534060949490 D9490
300534061518010 D8010
029 M0029
037 M0037
046 M0046
048 M0048


In [21]:
# Create the SWAN output file names, read SWAN model data, and populate bulk statistics arrays

dt = []
time = []
did = []
lat= []
lon= []
hsmod = []
hsobs = []
tpmod = []
tpobs = []
mdirobs  = []
mdirsobs = []
mdirmod  = []
mdirsmod = []

start_time = datetime(2023,8,30,11,35,0)
end_time = datetime(2023,8,30,12,15,0)

igood=0
icount=0
for dtype in drifter_types :
    drifter_data = drifters[dtype]
    for id in drifter_data.keys():

        # Some rows have times with other data but no wave info (Spotters only)
        only_waves = drifter_data[id]['energy_density'].notnull()

        ipt = 0 # index must stay below 1000 or file names will be too long
        for index, row in drifter_data[id][only_waves].loc['2023-08-29 1200':'2023-08-30 2300'].iterrows():
            
            print( (datetime.fromtimestamp(index)).between_time(start_time, end_time, include_start=True, include_end=False) )
           
            fn = "{}{:03d}.spc2d".format( snames[icount], ipt)
            pathname = os.path.join(MODEL_DIRECTORY, fn)

            index.dt.hour.between(8, 21, inclusive=True)
            if Path(pathname).is_file():
                # drifter obs
                igood+=1
                print( index, pathname )
                did.append( snames[icount] )
                dt.append( index )
                time.append( index.strftime('%Y%m%d.%H%m') )
                lat.append( row['latitude'] )
                lon.append( row['longitude'] )
                hsobs = ( row['significant_height'] )
                tpobs.append( row['peak_period'] )
                mdirobs.append( row['mean_direction'] )
                mdirsobs.append( row['mean_directional_spread'] )
                f = row['frequency']
                S = row['energy_density']
                a1 = row['a1']
                b1 = row['b1']
                a2 = row['a2']
                b2 = row['b2']
                # bulk parameters from f, S, a1, b1, etc.
                sigf =( calc_sigmaf_1d( S, f ) )
                sprd1 = ( calc_spread1_a1b1( a1, b1 ) )
                dm = calc_dirmf_a1b1( a1, b1)+180.
                dm[dm>=360]=dm[dm>=360]-360.
                
                # model output
                df = read_swan( pathname )
                hsmod =( np.squeeze( df.efth.spec.hs().values ) )
                tpmod.append( np.squeeze( df.efth.spec.tp().values ) )
                mdirmod.append(  np.squeeze( df.efth.spec.dpm().values ) )
                mdirsmod.append( np.squeeze( df.efth.spec.dspr().values ) )
                
                fs = df['freq'].values
                dirs = df['dir'].values
                # # flip the directions, so now directions are where waves come from
                dirs = dirs+180
                dirs[dirs>=360.]=dirs[dirs>=360.]-360.

                directional_bin_width_deg = dirs[2]-dirs[1]
                dirs_r = (np.pi/180.)*dirs
                spec2d = np.squeeze( df.efth.values ) 
                
                # This routine is from Isabel with a1 and b2 switched, and sign of a2 reversed.
                Ss, b1s, a2s, a1s, b2s = to_Fourier( spec2d, fs, dirs_r, directional_bin_width_deg, faxis=0, daxis=1 )
                a2s = -a2s

                # Almost no energy in high frequencies, so truncate
                igood = np.argwhere(Ss>1.e-8)
                fs = np.squeeze(fs[igood])
                Ss = np.squeeze(Ss[igood])
                a1s = np.squeeze(a1s[igood])
                a2s = np.squeeze(a2s[igood])
                b1s = np.squeeze(b1s[igood])
                b2s = np.squeeze(b2s[igood])

                # statistics for SWAN
                sprd1s = calc_spread1_a1b1( a1s, b1s )
                dms = calc_dirmf_a1b1( a1s, b1s)+180
                Hss = calc_Hs_1d( Ss, fs)
                print('SWAN Hs', hsmod, Hss, 'Hsobs: ', hsobs)

                # plot statistics
                fig, ax = plt.subplots( 3, 1, sharex=True )
                ax[0].plot(f, S, label='Drifter')
                ax[0].plot(fs, Ss, label='SWAN' )
                ax[0].set_ylabel(r'E (m$^2$/Hz)', fontsize=8)
                ax[1].plot(f, sprd1)
                ax[1].plot(fs, sprd1s)
                ax[1].set_ylabel('Dir. Spread (deg)', fontsize=8)
                ax[2].plot(f,dm)
                ax[2].plot(fs,dms)
                ax[2].set_ylabel('Dir. from (deg)', fontsize=8)
                ax[2].set_xlabel('Frequency (Hz)')
                ax[0].legend()
                #plt.savefig('spotter31232_swan_f_vs_dir.png', dpi=200, bbox_inches='tight')
               
            ipt += 1           
            
        icount +=1

TypeError: an integer is required (got type Timestamp)

In [14]:
Ss

array([1.13717043e-06, 8.52877825e-06, 3.46836982e-05, 1.10305532e-04,
       3.54797175e-04, 1.06041143e-03, 1.86268517e-03, 1.45444098e-03,
       3.60085018e-03, 1.02146334e-02, 6.90035019e-03, 1.11277813e-02,
       2.31152634e-02, 3.56571161e-02, 4.01625854e-02, 2.62851260e-02,
       1.26510211e-02, 6.73034321e-03, 3.77768018e-03, 1.93773842e-03,
       8.64818115e-04])